# 1. TrOCR 훈련 이미지 생성

--- 

0번 모듈에서 생성한 `ocr_dataset_poc.csv`를 사용하여 OCR 훈련 이미지를 생성합니다. 훈련 이미지 생성은 오픈 소스로 공개되어 있는 TextRecognitionDataGenerator를 사용합니다.

- 참조: https://github.com/Belval/TextRecognitionDataGenerator

In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=aa79590ae292e442b5ac1e139d2342f97bdb15aabf86d5b462ac13d45e7584c4
  Stored in directory: /home/ec2-user/.cache/pip/wheels/76/d9/1c/4059e4887aaf06a2481aec77e84f3b9f9a010e2b6ad523b95b
Successfully built wikipedia


In [2]:
import pandas as pd
import multiprocessing

df = pd.read_csv('ocr_dataset_poc.csv')
display(df.sample(n=10, random_state=42))
df['document'].to_csv('ocr_dataset_poc.txt', header=False, index=False)

,document,category
398118,누리호 발사 성공을 축하합니다,news
447804,그동안 두 자릿수였던 수출 증가율이 지난달 한 자릿수로 떨어지는 등 둔화세가 본격화...,news
464420,동원참치 캔 개발·생산 경험이 있는 동원시스템즈는 차별화된 기술을 적용한 4680 ...,news
534239,봉고 III EV 냉동탑차는 135kW 모터와 588kWh 배터리를 탑재해 완충 시...,news
235107,아 점깍은이유는 네이버가 영화 분류를 코믹으로 안하고 공포에 집어넣은 실수 때문에,nsmc
618045,유경준 의원은 개회사에서 “전 세계적으로 성장률 저하와 물가 상승이 동반되는 스테그...,news
633123,사랑은 변하고 사람은 안 변해요,chatbot
20017,꼴깝 떨고 앉았있네 별 시덥잖은 의미,nsmc
184000,청춘의지표는 여기서부터 시작이아닐까 싶다,nsmc
115726,차화연의 빛나는 미모,nsmc


전체 데이터셋은 60만건을 초과하지만, 핸즈온 시에는 1000건의 샘플만 사용합니다.

In [3]:
df['category'].value_counts()

news       333690
nsmc       284530
chatbot     19181
Name: category, dtype: int64

In [4]:
FULL_TRAINING = False

if not FULL_TRAINING:
    df = df.sample(n=1000)

In [5]:
num_cores = multiprocessing.cpu_count()
dataset_dir = 'train'
num_samples = len(df)

!rm -rf {dataset_dir}
!python3 ./trdg/run.py -i ocr_dataset_poc.txt -w 5 -t {num_cores} -f 64 -l ko -c {num_samples} -na 2 --output_dir {dataset_dir}
!cp {dataset_dir}/labels.txt ./

Missing modules for handwritten text generation.
100%|█████████████████████████████████████| 1000/1000 [00:00<00:00, 1058.05it/s]


선택적으로 랜덤하게 생성한 문장을 훈련 데이터셋에 추가합니다. 좀 더 다양한 형태의 데이터로 훈련해야 할 때 참고하세요. 

In [ ]:
# rand_dataset_dir = 'rand_train'
# num_rand_samples = 500000

# !rm -rf {rand_dataset_dir}
# !python3 ./trdg/run.py -w 5 -t {num_cores} -f 64 -l ko -c {num_rand_samples} -na 2 --output_dir {rand_dataset_dir}
# !cp {rand_dataset_dir}/labels.txt ./rand_labels.txt